In [ ]:
# Use NED Object Lookup api to fetch basic object information for given object name
# Documentation: http://vo.ned.ipac.caltech.edu/ui/Documents/ObjectLookup

import json
import requests
import urllib.parse

object_name  = 'cyg a'
encoded_name = urllib.parse.quote_plus(object_name)   # need to encode special characters in object name 

NED_object_lookup  = "http://ned.ipac.caltech.edu/srs/ObjectLookup?"
object_name_packet = 'json=' + json.dumps({"name":{"v":encoded_name}}, separators=(',',':'))
NED_object_lookup_response = requests.post(NED_object_lookup, data = object_name_packet)
if NED_object_lookup_response.status_code == 200:
    ned_object_basic_info = json.loads(NED_object_lookup_response.content)
    print(json.dumps(ned_object_basic_info, sort_keys=True,indent=4, separators=(',', ': ')))


In [ ]:
# Using NED TAP service with previously retrieved NED object position to perform 0.2 arcmin cone search
# Documentation: https://ned.ipac.caltech.edu/tap/

ned_tab     = 'https://ned.ipac.caltech.edu/tap'
table_name  = 'ned_objdir'
columns     = 'prefname,ra,dec,z,zflag'
coord_sys   = 'J2000'
coord_ra    = ned_object_basic_info['Preferred']['Position']['RA']   # From previous block
coord_dec   = ned_object_basic_info['Preferred']['Position']['Dec']  # From previous block
sr_deg      = 0.2 / 60              # 0.2 arcmin search radius
cone        = 'CONTAINS(POINT(\'' + str(coord_sys) + '\', ra, dec),CIRCLE(\'' + str(coord_sys) + '\',' + str(coord_ra) + ',' + str(coord_dec) + ',' + str(sr_deg) + ' ))=1'
query       = 'SELECT ' + columns + ' FROM ' + table_name + ' WHERE ' + cone

# Using astroquery

from astroquery.utils.tap.core import TapPlus
ned = TapPlus(url=ned_tab)
job = ned.launch_job_async(query)
out = job.get_results()
out.pprint()

# Using PyVO

import pyvo as vo
ned_TAP = vo.dal.TAPService(ned_tab)
ned_out = ned_TAP.search(query)
ned_out.table

In [ ]:
# Using NED TAP service to perform polygon seach for object count arount M51
# Documentation: https://ned.ipac.caltech.edu/tap/

ned_tab     = 'https://ned.ipac.caltech.edu/tap'
table_name  = 'ned_objdir'
columns     = 'count(*)'
coord_sys   = 'J2000'
polygon     = 'CONTAINS(POINT(\'' + str(coord_sys) + '\',ra,dec),POLYGON(\'' + str(coord_sys) + '\',202.333,47.033,202.667,47.033,202.667,47.367,202.333,47.367))=1'
query       = 'SELECT ' + columns + ' FROM ' + table_name + ' WHERE ' + polygon

# Using astroquery

from astroquery.utils.tap.core import TapPlus
ned = TapPlus(url=ned_tab)
job = ned.launch_job_async(query)
out = job.get_results()
out.pprint()

# Using PyVO

import pyvo as vo
ned_TAP = vo.dal.TAPService(ned_tab)
ned_out = ned_TAP.search(query)
ned_out.table

In [ ]:
# Using NED TAP service to perform polygon seach for object count arount M51 for objects within |dcz| = 500 km/s
# Documentation: https://ned.ipac.caltech.edu/tap/

ned_tab     = 'https://ned.ipac.caltech.edu/tap'
table_name  = 'ned_objdir'
columns     = 'count(*)'
coord_sys   = 'J2000'
polygon     = 'CONTAINS(POINT(\'' + str(coord_sys) + '\',ra,dec),POLYGON(\'' + str(coord_sys) + '\',202.333,47.033,202.667,47.033,202.667,47.367,202.333,47.367))=1'
z_range     = 'z BETWEEN 0.000333 AND 0.003667'
query       = 'SELECT ' + columns + ' FROM ' + table_name + ' WHERE ' + polygon + ' AND ' + z_range

# Using astroquery

from astroquery.utils.tap.core import TapPlus
ned = TapPlus(url=ned_tab)
job = ned.launch_job_async(query)
out = job.get_results()
out.pprint()

# Using PyVO

import pyvo as vo
ned_TAP = vo.dal.TAPService(ned_tab)
ned_out = ned_TAP.search(query)
ned_out.table

In [ ]:
# Using NED SED service 

import io
import requests
import urllib.parse
import matplotlib.pyplot as plt
from astropy.table import Table

object_names = ('arp220','3c273','BL Lac')
color        = ('r', 'b', 'g')

NED_sed  = 'http://vo.ned.ipac.caltech.edu/services/accessSED?'

i=0
for name in object_names:
    paramters = { 'REQUEST':'getData','TARGETNAME':name }
    NED_sed_response = requests.get(NED_sed, params = paramters)
    if NED_sed_response.status_code == 200:
        ned_data_table = Table.read(io.BytesIO(NED_sed_response.content))
        x=ned_data_table['DataSpectralValue']
        y=ned_data_table['DataFluxValue']
        plt.plot(x, y, color[i])
        j=-i*15
        plt.annotate(name, xy=(x[j],y[j]), xytext=(x[j]*100,y[j]*100),arrowprops=dict(facecolor=color[i], shrink=0.05),color=color[i])
        i+=1
plt.xlim(1e6,1e20)
plt.ylim(1e-9,1e3)
plt.yscale('log')
plt.xscale('log')
plt.show()
